In [ ]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.classifier import StackingClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics

import My_Transformers
# Global var
My_Transformers.TRAIN_ROW_COUNT = 7613
from My_Transformers import *
# from Word2Vec_Transformer import WVEmbeddingTransformer, Text_cleaner_transformer, EMB_VOCAB_SET, WV_EMBEDDINGS
import warnings
warnings.filterwarnings("ignore")

def pickleSave (obj, file):
    
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):
    return pickle.load (open (file, 'rb'))

def init ():

    train_file = "Data/all.csv"
    df = pd.read_csv (train_file)
    # df = df[0:20]
    df['text'] = df.text.astype (str)
    df['keyword'] = df.keyword.astype (str)
    # keywords_train = list (df['keyword'])
    # LB = LabelBinarizer ()
    # LB.fit (keywords_train)
    # LB_loc = LabelBinarizer ()
    # df['location'] = LB_loc.fit_transform (list (pd.isnull (df['location'])))
    Xstr = df["text"]
    y = df["target"]
    return df, Xstr, y

def train (Xstr, y, isCV=False):

    Xstr_transformer = Pipeline ([

        ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
        ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
        ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
        ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
        ("pca_nmf_select",          PCA_NMF_TrainTest_FeatureTransformer (isSparseOut=True))
    ])
    X = Xstr_transformer.fit_transform (Xstr, y)
    X = X.tocsr ()
    pickleSave (X, 'X.bin')
    # X = unpickle ('X.bin')
    X.shape

    pickleSave (Xstr_transformer, 'Xstr_transformer.bin')
    # Xstr_transformer = unpickle ('Xstr_transformer.bin')
    # pickleSave (rf_importance_feature_selector, 'rf_importance_feature_selector.bin')

    params_lsvc  = { 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'},                                                                   'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }
    params_logit = { 'name': 'Logit', 'params': {'penalty': 'l2', 'class_weight': 'balanced'},                                                'param_grid': {'C': [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10], 'fit_intercept': [True, False]} }
    params_rf    = { 'name': 'RF',    'params': {'class_weight': 'balanced'} }
    params_xgb   = { 'name': 'Xgb',   'param_grid':{'max_depth': [3, 5], 'learning_rate':[1.5, 3]} }
    params_catb  = { 'name': 'Catb',  'isCV': isCV }

    params    = [params_lsvc, params_logit, params_catb]
    bclfs     = []
    cv_scores = []
    for param in params:

        bclf = Best_clf_cv_transformer (param)
        bclf.fit (X, y)
        bclfs.append ((bclf, bclf.get_cv_score()))
        cv_scores.append (bclf.get_cv_score())
    return X, Xstr_transformer, bclfs, params

def predict (bclfs, Xstr_transformer):

    test_file = "Data/test.csv"
    testDF = pd.read_csv (test_file)
    # testDF = testDF[0:10]
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    X_test_str = testDF["text"]
    testDF.shape

    X_test = Xstr_transformer.transform (X_test_str)
    X_test = X_test.tocsr ()
    pickleSave (X_test, 'X_test.bin')
    X_test.shape
    for bclf in bclfs:

        bclf = bclf[0]
        predictions = bclf.predict (X_test)
        testDF['target'] = predictions
        testDF['target'] = testDF.target.astype (int)
        submitDF = testDF[['id', 'target']]
        submitDF.to_csv ('submission_v4_' + bclf.name + '.csv', index=False)
    return

def doVoting (X, y, params, bclfs):

    # voting classifier
    a = 1.0 / np.sum(y)
    b = 1.0 / (len(y) - np.sum(y))
    w1 = a / (a + b)
    w0 = b / (a + b)
    sample_weight = []
    for i in y:
        if i:
            sample_weight.append(w1)
        else:
            sample_weight.append(w0)

    voting_candidates = []
    for param in params:
        param['isCV'] = False
        bclf = Best_clf_cv_transformer (param)
        voting_candidates.append ((bclf.name, bclf))
    voting_clf = VotingClassifier (estimators=voting_candidates, voting='soft', weights=[1, 1, 1.5])
    voting_clf.name = 'Voting'
    train_len = len(y[pd.isnull(y) == False])
    X_train, y_train = X[:train_len], y[:train_len]
    # print ('CV for VotingClassifier:')
    # cv_results = cross_validate (voting_clf, X_train, y_train, cv=5)  #, fit_params={'sample_weight': sample_weight})
    voting_clf_cv_score = 0  # np.mean (cv_results['test_score'])
    print ('Voting_clf cv-score =', voting_clf_cv_score)
    voting_clf.fit(X_train, y_train)

    # save the trained classifiers
    bclfs.append((voting_clf, voting_clf_cv_score))
    pickleSave(bclfs, 'bclfs.bin')
    # bclfs = unpickle ('bclfs.bin')
    # Choose the bclf with the best cv_score:
    bclf = max(bclfs, key=lambda i: i[1])[0]
    print ('bclf =', bclf)
    # cv_results
    return bclfs

def main (threshold=0.9, maxIterCount=5):

    X, params, bclfs = None, None, None
    df, Xstr, y = init ()
    newTrainAddCount = 1
    iterCount = 0
    while newTrainAddCount > 0:

        threshold = threshold - iterCount * 0.02
        iterCount += 1
        if iterCount > maxIterCount:
            break
        X, Xstr_transformer, bclfs, params = train (Xstr, y, False)
        bclf = bclfs[2][0]
        newTrainAddCount = 0
        for idx, row in df.iterrows():
            if not pd.isnull (row['target']):

                continue
            print ('considering idx =', idx)
            x_str = row['text']
            x  = Xstr_transformer.transform ([x_str]).reshape (1, -1)
            pr = bclf.predict_proba (x)[0]
            p  = 0
            if pr[0] > 0.5:
                pr = pr[0]
                p  = 0
            else:
                pr = pr[1]
                p  = 1
            if pr >= threshold:
                # add in the training set
                y[idx] = p
                df.at[idx, 'target'] = p
                newTrainAddCount += 1
        # arrange df, Xstr, y such that 1st rows with y=0/1 then rows with y=NaN
        arrangedDF1 = df[pd.isnull(df['target']) == False]
        arrangedDF2 = df[pd.isnull(df['target'])]
        df = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
        Xstr = df['text']
        y = df['target']
    df.to_csv ('Data/all.csv', index=False)
    X = Xstr_transformer.transform (Xstr)
    bclfs = doVoting (X, y, params, bclfs)
    predict (bclfs, Xstr_transformer)
    # free memory
    df = None
    Xstr = None

    # CV for the bclfs on the original train rows
    print ('------------------- CV for the bclfs on the original train rows -------------------')
    X = X[:7613]
    y = y[:7613]
    cv_results = []
    for bclf in bclfs:

        bclf = bclf[0]
        cv_result = cross_validate (bclf, X, y, cv=5)
        cv_results.append ((bclf.name, cv_result))
    for cv_result in cv_results:

        clf_name = cv_result[0]
        cv_result = cv_result[1]
        cv_score = np.mean (cv_result['test_score'])
        print ()
        print (clf_name, cv_score)
        print (cv_result)
    return

if __name__=="__main__":

    main (threshold=0.9, maxIterCount=1)


# Training snippet:

    
LinearSVC with L1-based feature selection for X.shape = (10511, 131984)

--------------------- Best parameter (CV score=0.819):
    
{'feature_selection__estimator__tol': 0.0001}

New #features =  1249

LinearSVC with L1-based feature selection for X.shape = (10511, 780625)

--------------------- Best parameter (CV score=0.853):
    
{'feature_selection__estimator__tol': 0.001}

New #features =  1236

Find optimal PCA dims and the same no. of NMF features for X.shape = (10876, 1236)

--------------------- Best:  parameters = (0.99, 10) , CV = 0.8819354616869652
    
PCA dimensionality, explainedVarRatio =  1034 0.99

training LSvc for X.shape = (10511, 2068)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.882):
            
{'C': 10}

training Logit for X.shape = (10511, 2068)

Logit : Best_clf_cv_transformer: Best parameter (CV score=0.873):
            
{'C': 10, 'fit_intercept': True}

training Catb for X.shape = (10511, 2068)


# Findings: 5-fold CV on train data:
        
LSvc 0.8166336929240046

{'fit_time': array([4.14286661, 2.96390486, 2.84986711, 3.2749989 , 2.96900582]), 'score_time': array([0.03899908, 0.02418208, 0.0240612 , 0.02711415, 0.03301454]), 'test_score': array([0.82206172, 0.79054498, 0.80499015, 0.80749014, 0.85808147])}


Logit 0.7881285898310876

{'fit_time': array([0.452003  , 0.56100106, 0.55493712, 0.61006212, 0.55389619]), 'score_time': array([0.00509501, 0.00593638, 0.00600028, 0.0050025 , 0.00605416]), 'test_score': array([0.79579777, 0.77150361, 0.77413001, 0.78120894, 0.81800263])}


Catb 0.7760440654597097

{'fit_time': array([147.6999433 , 154.35827494, 149.82401252, 158.81929469,
       152.28349304]), 'score_time': array([0.15506792, 0.10985398, 0.13898897, 0.11499739, 0.09900331]), 'test_score': array([0.7800394 , 0.76165463, 0.76296783, 0.7542707 , 0.82128778])}


Voting 0.803103960904329

{'fit_time': array([136.75651956, 144.03056884, 149.86489344, 145.66331077,
       146.97007728]), 'score_time': array([0.13100529, 0.14399958, 0.1439662 , 0.14199662, 0.14699697]), 'test_score': array([0.80236376, 0.78726198, 0.78923178, 0.79040736, 0.84625493])}